In [1]:
from bert_logistic import prepare_data_for_model, read_texts_from_dir, train_and_evaluate
import pandas as pd
import numpy as np

/home/thangquang09/CODE/CTAI_MachineLearning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train"
test_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/test"
gt_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train.csv"
    

In [3]:
print("Loading data...")
df_train = read_texts_from_dir(train_path)
df_test = read_texts_from_dir(test_path)
df_train_gt = pd.read_csv(gt_path)
y_train = df_train_gt["real_text_id"].values
    

Loading data...
Number of directories: 95
Number of directories: 1068


In [4]:
print("Preparing training data...")
X_train, embedding_extractor = prepare_data_for_model(
    df_train, 
    fit_embedding=True, 
    model_name='allenai/scibert_scivocab_uncased'
)

Preparing training data...
Step 1: Preprocessing text...


Cleaning text: 100%|██████████| 95/95 [00:00<00:00, 980.07it/s]


Step 2: Extracting rule-based features...


Extracting rule-based features: 100%|██████████| 95/95 [00:00<00:00, 200.17it/s]


Step 3: Extracting statistical features...
Step 4: Extracting embedding features...
Loading embedding model: allenai/scibert_scivocab_uncased
Loaded as Transformers model
Extracting embedding features for file_1...


Extracting embeddings: 100%|██████████| 3/3 [00:17<00:00,  5.99s/it]


Extracting embedding features for file_2...


Extracting embeddings: 100%|██████████| 3/3 [00:17<00:00,  5.97s/it]

Step 5: Combining all features...
Final feature matrix shape: (95, 3116)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [7]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# Initialize CatBoostClassifier
catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=0)

# Train the model
catboost_model.fit(X_train, y_train)

# Predict on validation set
y_preds_val_catboost = catboost_model.predict(X_val)

# Calculate accuracy
acc_catboost = accuracy_score(y_val, y_preds_val_catboost)
print(f"Validation accuracy with CatBoost: {acc_catboost:.4f}")

Validation accuracy with CatBoost: 0.8421


In [ ]:
X_test, _ = prepare_data_for_model(
        df_test, 
        embedding_extractor=embedding_extractor, 
        fit_embedding=False
    )

In [8]:
from pathlib import Path

print("Predicting on test ...")
test_pred = catboost_model.predict(X_test)

# --- Build submission -------------------------------------------------
submission = pd.DataFrame({
    "id": df_test.index,
    "real_text_id": test_pred.astype(int)
}).sort_values("id")

save_path = Path("submission__catboost.csv")
submission.to_csv(save_path, index=False)
print(f"✅ Submission saved to {save_path.resolve()}")

Predicting on test ...
✅ Submission saved to /home/thangquang09/CODE/CTAI_MachineLearning/notebooks/submission__catboost.csv
